In [17]:
def get_pipeline(vuelo, zona, tipo):
    if vuelo.startswith('vuelo'):
        class_range = '[4:]'
    else:
        class_range = '[3:5]'
        
    return f'''{{
      "pipeline":[
        {{
          "type":"readers.pgpointcloud",
          "connection":"host='172.17.0.2' dbname='pointclouds' user='postgres' password='ventanuco' port='5432'",
          "table":"{vuelo}_zona{zona}",
          "column": "pa",
          "spatialreference":"EPSG:25830",
          "where": "pc_numpoints(pa) is not null"
        }},
        {{
          "type": "filters.range",
          "limits": "Classification{class_range}"
        }},
        {{
          "type":"filters.hag_dem",
          "raster": "MDT/MDT02-ETRS89-HU30-0174-2-COB2.tif"
        }},
        {{
          "type": "filters.range",
          "limits": "HeightAboveGround{tipo['range']}"
        }},
        {{
          "type":"writers.gdal",
          "filename":"FINALES/{vuelo}_zona{zona}_{tipo['nombre']}.tif",
          "gdaldriver":"GTiff",
          "output_type":"max",
          "resolution":"1.0",
          "dimension":"HeightAboveGround"
        }}
      ]
    }}'''

In [19]:
import pdal

for vuelo in ['lida2', 'lida3', 'vuelo1', 'vuelo2']:
    for zona in [2]:
        for tipo in [{'nombre': 'arbolado', 'range': '[3:]'}, {'nombre': 'matorral', 'range': '[0.2:3]'}]:
            pipeline = pdal.Pipeline(get_pipeline(vuelo, zona, tipo))
            pipeline.execute()

In [16]:
from pathlib import Path
from rasterio.merge import merge
import rasterio

path = Path(".")
files = list(path.glob(f"FINALES/temp/vuelo2_zona3_matorral_*"))

dest, output_transform = merge(files)

with rasterio.open(files[0]) as src:
    out_meta = src.meta.copy()
    out_meta.update(
        {
            "driver": "GTiff",
            "height": dest.shape[1],
            "width": dest.shape[2],
            "transform": output_transform
        }
    )

with rasterio.open(f"FINALES/vuelo_2_zona3_matorral.tif", "w", **out_meta) as fp:
    fp.write(dest)